# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


# Этап 1. Получение данных

Посмотрим на данные, предоставленные банком.

In [1]:
import pandas as pd
# df = pd.read_csv('Data_Scientist/Initial Data/data.csv')
df = pd.read_csv('/datasets/data.csv') # Код ревьюера
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем общую информацию о данных таблицы data.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Проблемы в данных:**
* Количество значений в столбцах различается. Это говорит о том, что в полях *days_employed* и *total_income* данных есть пропущенные значения.
* Поле *days_employed* имеет тип данных float64, потребуется преобразование в int64.
* Поле *education* содержит текст в разных регистрах.


**Вывод**

Каждая строка таблицы содержит информацию о возрасте, трудовом стаже, семейном положении, количестве детей, ежемесячном доходе и цели получения кредита. В данных есть проблемы, которые нужно устранить перед тем, как приступать к обработке.

# Этап 2. Предобработка данных

Исключим пропуски, проверим столбцы, а также проверим данные на наличие дубликатов.

Получим перечень названий столбцов:

In [3]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Названия столбцов устаривают, переименовывать не будем.

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Судя по одинаковому количеству пропущенных в полях *days_employed* и *total_income* значений, видимо, пропущены они в одних и тех же строках. Проверим, так ли это.

In [5]:
unemployed = df.loc[df.loc[:,'days_employed'].isnull() & df.loc[:,'total_income'].isnull()]
unemployed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Наше предположение подтвердилось. Это может говорить либо о проблеме в источнике статистических данных, либо о том, что официально люди не были трудоустроены, поэтому данные о стаже и ежемесячном доходе отсутствуют. Однако, в других полях есть данные, которые пригодятся для нашего исследования. Не будем удалять эти строки из выборки, тем более что их почти 10% от всего датафрейма.
Так как пропущены значения количественных переменных, то заменять их на строку неправильно. 

Создадим в датафрейме ещё одно поле 'employment_info', и заполним его значениями 'unknown', если инфомации нет и 'available' - если есть.

In [6]:
df['employment_info'] = df.loc[:,'days_employed'].isnull()
df['employment_info'] = df['employment_info'].replace(True,'unknown')
df['employment_info'] = df['employment_info'].replace(False,'available')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employment_info
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,available
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,available
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,available
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,available
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,available
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,available
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,available
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,available
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,available
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,available


Сгруппируем данные по полю 'employment_info', чтобы посмотреть, чем заняты люди с неизвестным стажем и ежемесячным доходом.

In [7]:
employment_grouped=df.groupby('employment_info')['income_type']
employment_grouped.value_counts()

employment_info  income_type    
available        сотрудник          10014
                 компаньон           4577
                 пенсионер           3443
                 госслужащий         1312
                 безработный            2
                 в декрете              1
                 предприниматель        1
                 студент                1
unknown          сотрудник           1105
                 компаньон            508
                 пенсионер            413
                 госслужащий          147
                 предприниматель        1
Name: income_type, dtype: int64

Итак, у нас есть сотрудники, пенсионеры и госслужащие с неизвестным стажем и зарплатой (пенсией). И ни одного блогера. То есть безработными их не назовешь. 

**Вывод:** значения в полях 'days_employed' и 'total_income' отсутствуют из-за проблемы в источнике данных.

- **Проверим столбец 'children' на наличие нереалистичных данных**

In [8]:
children_grouped=df.groupby('children')['children']
children_grouped.count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Значения -1 и 20 - явные ошибки. Можно было бы удалить эти данные, т.к. в сумме это 0,57% от выборки. Но можно также предположить, что данные о количестве детей заполнялись вручную и были допущены опечатки. Скорее всего имелось в виду 1 и 2 ребенка соответственно. Нужно указать на эти ошибки команде разработчиков софта, и порекомендовать сделать выпадающий список для ввода количества детей.
Заменим неверные значения на адекватные. Сделаем проверку.

In [9]:
df['children'] = df['children'].replace(-1,1)
df['children'] = df['children'].replace(20,2)

In [10]:
children_grouped=df.groupby('children')['children']
children_grouped.count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Итог.** Аномалии в столбце 'children' устранены.

- **Проверим, есть ли аномалии в данных в столбце 'gender'**

In [11]:
gender_grouped=df.groupby('gender')['gender']
gender_grouped.count()

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
df.loc[df.loc[:,'gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employment_info
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,available


Аномалия найдена. Так как пол в нашем исследовании не так важен, а запись всего одна, просто определим этого персонажа к любому полу. Сделаем проверку.

In [13]:
df['gender'] = df['gender'].replace('XNA','F')

In [14]:
gender_grouped=df.groupby('gender')['gender']
gender_grouped.count()

gender
F    14237
M     7288
Name: gender, dtype: int64

**Итог.** Мы избавились от аномалии в данных столбца 'gender'. 

- **Проверим поле 'family_status' на наличие аномалий**

In [15]:
family_status_grouped=df.groupby('family_status')['family_status']
family_status_grouped.count()

family_status
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
Name: family_status, dtype: int64

Аномалий нет, однако одна из категорий имеет символ в верхнем регистре. Для красоты приведем его к нижнему. Сделаем проверку.

In [16]:
df['family_status'] = df['family_status'].str.lower()
family_status_grouped=df.groupby('family_status')['family_status']
family_status_grouped.count()

family_status
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
не женат / не замужем     2813
Name: family_status, dtype: int64

**Итог.** Все строки поля 'family_status' теперь в одном регистре.

- **Проверим поле 'education' на наличие аномалий**

В поле 'education' содержатся строки в разном регистре. Такое бывает, когда пользователь вводит значения вручную, а не выбирает из списка. Либо данные были заполнены из разных источников, в которых справочники разные. Приведем строки к нижнему регистру.

In [17]:
df['education'] = df['education'].str.lower()
education_grouped = df.groupby('education')['education']
education_grouped.count()

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

**Итог.** Категории образования нормализованы.

- **Проверим поле 'income_type' на наличие аномалий**

In [18]:
income_type_grouped=df.groupby('income_type')['income_type']
income_type_grouped.count()

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

**Итог.** Аномалий в поле 'income_type' не обнаружено

- **Удалим дубликаты**

Найдём суммарное количество дубликатов, применив метод нахождения дубликатов к датафрейму с последующим суммированием.

In [19]:
df.duplicated().sum()

71

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».
Удалим дубликаты и сделаем проверку.

In [20]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Мы удалили все дубликаты.

- **Проверим поле 'days_employed' на наличие аномалий**

В поле 'days_employed' есть отрицательные значения. Возможно, имеет место проблема при выгрузке данных. Так как стаж не может быть отрицательным, напишем функцию 'neg_to_pos', и применим её к отрицательным значениям в поле 'days_employed' (там, где они есть).

In [21]:
def neg_to_pos(value):
    if value < 0:
        result = value*(-1)
    else:
        result = value
    return result

In [22]:
df['days_employed'] = df['days_employed'].apply(neg_to_pos)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employment_info
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,available
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,available
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,available
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,available
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,available
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,available
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,available
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,available
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,available
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,available


Для удобства дальнейшей обработки заменим все NaN на 0 методом fillna(), чтобы вдальнейшем применить математические методы. Также применим метод astype() с аргументом 'int' для столбцов 'days_employed' и 'total_income', чтобы избавиться от дробной части чисел типа 'float'.

In [23]:
df=df.fillna(0)
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Отсортируем таблицу по возрасту в обратном порядке, чтобы узнать, каков максимальный возраст заёмщика.

In [24]:
df_sorted = df.sort_values(by='dob_years', ascending=False)
#df.info()
df_sorted.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employment_info
8870,0,1678,75,среднее,1,вдовец / вдова,2,F,госслужащий,0,153282,заняться образованием,available
12297,0,1729,74,среднее,1,женат / замужем,0,M,компаньон,0,124830,покупка жилой недвижимости,available
19584,0,380150,74,среднее,1,вдовец / вдова,2,F,пенсионер,0,45089,приобретение автомобиля,available
2557,0,372861,74,среднее,1,женат / замужем,0,F,пенсионер,0,42927,автомобили,available
11513,0,6682,74,среднее,1,гражданский брак,1,F,сотрудник,0,98945,сыграть свадьбу,available
4891,0,341528,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935,покупка своего жилья,available
3458,0,344623,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754,операции со своей недвижимостью,available
19329,0,372290,73,начальное,3,гражданский брак,1,F,пенсионер,0,100166,на проведение свадьбы,available
1826,0,368375,73,среднее,1,вдовец / вдова,2,F,пенсионер,0,74284,покупка жилой недвижимости,available
229,0,336747,73,среднее,1,вдовец / вдова,2,F,пенсионер,0,136897,сделка с подержанным автомобилем,available


Итак, если взять самого старшего заёмщика, и предположить, что он работал с 18 лет без выходных и отпуска все 57 лет, то получится 200805 дней. Однако, судя по данным, эта госслужащая проработала 4 с половиной года, при этом имея высокий ежемесячный доход. Как-то странно. Посчитаем количество больших значений в столбце 'days_employed'.

In [25]:
df.loc[df.loc[:,'days_employed'] >20000]['days_employed'].count()

3445

Итого как минимум 16% датафрейма имеют нереалистичные значения.

Посмотрим, сколько зарабатывают самые неопытные работники со стажем от 1 до 365 дней

In [26]:
not_expiriensed_count = df.loc[(df.loc[:,'days_employed'] >0)&(df.loc[:,'days_employed'] <365)]['total_income'].count()
not_expiriensed_income = df.loc[(df.loc[:,'days_employed'] >0)&(df.loc[:,'days_employed'] <365)]['total_income'].median()
print(not_expiriensed_count)
print(not_expiriensed_income)

1827
142057.0


Если это значение в рублях, то новички неплохо зарабатывают. Может быть, все заёмщики достаточно высокооплачиваемы? 
Найдем среднюю зарплату всей выборки по медиане:

In [27]:
total_income_median = df['total_income'].median()
total_income_median

135781.0

Получается новички имеют среднюю зарплату. Таким образом, можно сделать вывод о том, что ещё как минимум 7% датафрейма имеют нереалистичные значения.
Следовательно данные в столбце 'days_employed' не релевантные. К счастью, для решения поставленной задачи это поле не нужно.

**Вывод:** в финальные таблицы мы не будем включать поле 'days_employed', и укажем команде, что необходимо разобраться в причинах появления "плохого" столбца.

- **Проверим поле 'purpose_names' на наличие аномалий**

Методом value_counts() определим уникальные значения столбца 'purpose' и их количество.

In [28]:
#employment_grouped=df.groupby('purpose').count()
purpose_names=df['purpose'].value_counts()
purpose_names

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Сохраним количество строк таблицы 'purpose_names' в переменной purpose_names_len.

In [29]:
purpose_names_len=len(purpose_names)
purpose_names_len

38

In [30]:
purpose_names.dtype

dtype('int64')

Очевидно, цель заполнялась вручную, и данные в полученном списке не пригодны для каких-то выводов.
Нам понадобится лемматизация этого списка.

В полученном нами объекте Series индексами являются наименования целей займа, а их количество - значениями.
Превратим purpose_names в датафрейм, сбросив индексы методом reset_index().

In [31]:
purpose_names = purpose_names.reset_index()
purpose_names.head()

,index,purpose
0,свадьба,791
1,на проведение свадьбы,768
2,сыграть свадьбу,765
3,операции с недвижимостью,675
4,покупка коммерческой недвижимости,661


Значения в полученном столбце 'index' нам нужны. А вот название колонки нужно заменить на значимое, а именно на 'purpose'. 
Значения в старом столбце 'purpose' не нужны, но имеют тип данных int, что нам на руку, потому что мы можем использовать его для присвоения в дальнейшем идентификаторов новых категорий кредита. Назовем колонку 'purpose_id'.

In [32]:
purpose_names.columns = ['purpose','purpose_id']
purpose_names.head()

,purpose,purpose_id
0,свадьба,791
1,на проведение свадьбы,768
2,сыграть свадьбу,765
3,операции с недвижимостью,675
4,покупка коммерческой недвижимости,661


Создадим список из строк столбца 'purpose' и выведем его на экран.

In [33]:
purpose_names_list=purpose_names['purpose'].tolist()
purpose_names_list

['свадьба',
 'на проведение свадьбы',
 'сыграть свадьбу',
 'операции с недвижимостью',
 'покупка коммерческой недвижимости',
 'операции с жильем',
 'покупка жилья для сдачи',
 'операции с коммерческой недвижимостью',
 'жилье',
 'покупка жилья',
 'покупка жилья для семьи',
 'строительство собственной недвижимости',
 'недвижимость',
 'операции со своей недвижимостью',
 'строительство жилой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'строительство недвижимости',
 'ремонт жилью',
 'покупка жилой недвижимости',
 'на покупку своего автомобиля',
 'заняться высшим образованием',
 'автомобиль',
 'сделка с подержанным автомобилем',
 'на покупку подержанного автомобиля',
 'свой автомобиль',
 'автомобили',
 'на покупку автомобиля',
 'приобретение автомобиля',
 'дополнительное образование',
 'сделка с автомобилем',
 'высшее образование',
 'образование',
 'получение дополнительного образования',
 'получение образования',
 'профильное образование',
 'получение высшего образован

Превратим список в строку. Напишем функцию ListToString(lst), которая использует функцию .join(). 
Передадим этой функции в качестве аргумента список purpose_names_list

In [34]:
def ListToString(lst):  
    
    # Создаём пустую строку 
    str1 = " " 
    
    # Возвращаем строку
    return (str1.join(lst)) 

purpose_names_string = ListToString(purpose_names_list) 
purpose_names_string

'свадьба на проведение свадьбы сыграть свадьбу операции с недвижимостью покупка коммерческой недвижимости операции с жильем покупка жилья для сдачи операции с коммерческой недвижимостью жилье покупка жилья покупка жилья для семьи строительство собственной недвижимости недвижимость операции со своей недвижимостью строительство жилой недвижимости покупка недвижимости покупка своего жилья строительство недвижимости ремонт жилью покупка жилой недвижимости на покупку своего автомобиля заняться высшим образованием автомобиль сделка с подержанным автомобилем на покупку подержанного автомобиля свой автомобиль автомобили на покупку автомобиля приобретение автомобиля дополнительное образование сделка с автомобилем высшее образование образование получение дополнительного образования получение образования профильное образование получение высшего образования заняться образованием'

Посчитаем число упоминаний лемматизированных слов в строке purpose_names_string.

In [35]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

lemmas = m.lemmatize(purpose_names_string)

print(Counter(lemmas))

Counter({' ': 96, 'недвижимость': 10, 'покупка': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'на': 4, 'операция': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'высокий': 3, 'получение': 3, 'коммерческий': 2, 'для': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'подержать': 2, 'дополнительный': 2, 'проведение': 1, 'сыграть': 1, 'сдача': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'ремонт': 1, 'приобретение': 1, 'профильный': 1, '\n': 1})


Чаще всего упоминаются слова: недвижимость, покупка, автомобили, образование, жильё, свадьба. 
Так как слово 'покупка' используется вместе с другими словами, не будем выделять его в категорию. Слово 'жилье' будем интерпретировать, как ту же 'недвижимость'.

Создадим словарь с категориями целей займа, который пригодится нам в дальнейшем.

In [36]:
lst = [
    ['свадьба',1],
    ['недвижимость',2],
    ['автомобиль',3],
    ['образование',4]
]

entries = ['target','purpose_id']

purpose_dict = pd.DataFrame(data = lst, columns = entries)
purpose_dict

,target,purpose_id
0,свадьба,1
1,недвижимость,2
2,автомобиль,3
3,образование,4


Напишем функцию purpose_cat(text), которая возвращает id категории в зависимости от строки, полученной при лемматизации.
Сделаем проверку.

In [37]:
def purpose_cat(text):
    if 'свадьба' in text:
        cat = 1
    elif  'недвижимость' in text:
        cat = 2
    elif  'жилье' in text:
        cat = 2 
    elif  'автомобиль' in text:
        cat = 3
    elif  'образование' in text:
        cat = 4    
    return cat
purpose_cat(['свадьба', '\n'])

1

Напишем функцию lemmatization(lst), которая выводит на экран cписок списков лемм и возвращает результирующую категорию.
Сделаем проверку.

In [38]:
from pymystem3 import Mystem
m = Mystem()

def lemmatization(lst):
    lemmas = m.lemmatize(lst)
    sign = purpose_cat(lemmas)
    print(lemmas)
    return(sign)
lemmatization(purpose_names['purpose'][2])

['сыграть', ' ', 'свадьба', '\n']


1

Применим функцию lemmatization(lst) к столбцу датафрейма purpose_names 'purpose'. Результат перезапишем в столбец 'purpose_id'.

In [39]:
try:
    purpose_names['purpose_id'] = purpose_names['purpose'].apply(lemmatization)
except:
    print()
    print('ERROR: Дополни список в  purpose_cat(value)')

['свадьба', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['жилье', '\n']
['покупка', ' ', 'жилье', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'свой', ' ', 'жилье', '\n']
['строительство', ' ', 'недвижимость', '\n']
['ремонт', ' ', 'жилье', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['заниматься', ' ', 'высоки

Выведем таблицу purpose_names.

In [40]:
purpose_names

,purpose,purpose_id
0,свадьба,1
1,на проведение свадьбы,1
2,сыграть свадьбу,1
3,операции с недвижимостью,2
4,покупка коммерческой недвижимости,2
5,операции с жильем,2
6,покупка жилья для сдачи,2
7,операции с коммерческой недвижимостью,2
8,жилье,2
9,покупка жилья,2


Объединим таблицу df с таблицей purpose_names.

In [41]:
df = df.merge(purpose_names, on='purpose', how='left')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employment_info,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,available,2
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,available,3
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,available,2
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,available,4
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,available,1


Оценим, какого возраста заёмщиков больше всего.

In [42]:
df['dob_years'].value_counts()

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Категоризируем данные о возрасте клиентов так:
- Клиенты до 35 попадают в категорию «молодые»;
- Клиенты от 35 до 64 лет — категория «взрослые»;
- Клиенты старше 65 лет принадлежат к категории «пожилые».

Напишем функцию age_group. Сделаем проверку

In [43]:
def age_group(age):
        """
        Возвращает id возрастной группы по значению возраста age, используя правила:
        - 'молодые' при значении age <= 35 лет
        - 'взрослые' при значениии age более 35 и менее 64, включая 64
        - 'пожилые' во всех остальных случаях
        """

        if age <= 35:
                return  1 #'молодые'
        if age <= 64:
                return  2 #'взрослые'
        return 3 #'пожилые' 
print(age_group(35))
print(age_group(36))
print(age_group(65))

1
2
3


In [44]:
lst = [
    ['молодые',1],
    ['взрослые',2],
    ['пожилые',3]
]

entries = ['age_group','age_group_id']

age_group_dict = pd.DataFrame(data = lst, columns = entries)
age_group_dict

,age_group,age_group_id
0,молодые,1
1,взрослые,2
2,пожилые,3


Категоризируем данные о доходе клиентов так:
- Отношение total_income/total_income_median < 0.5 попадает в категорию «низкий»;
- Отношение total_income/total_income_median < 1.5 попадают в категорию «средний»;
- Отношение total_income/total_income_median >= 1.5 попадают в категорию «высокий»;

Напишем функцию income_group. Сделаем проверку

In [45]:
def income_group(income):

        if income/total_income_median < 0.5:
                return  1 #'низкий'
        if income/total_income_median < 1.5:
                return  2 #'средний'
        return 3 #'высокий' 
print(income_group(60000))
print(income_group(100000))
print(income_group(300000))

1
2
3


In [46]:
lst = [
    ['низкий',1],
    ['средний',2],
    ['высокий',3]
]

entries = ['income_group','income_group_id']

income_group_dict = pd.DataFrame(data = lst, columns = entries)
income_group_dict

,income_group,income_group_id
0,низкий,1
1,средний,2
2,высокий,3


Создадим в таблице df колонку 'age_group'. Вызовем метод apply() для колонки 'dob_years' и применим к нему функцию age_group.
Создадим в таблице df колонку 'income_group_id'. Вызовем метод apply() для колонки 'total_income' и применим к нему функцию income_group.
Выведем верхние 5 строк таблицы df.

In [47]:
df['age_group_id'] = df['total_income'].apply(age_group)
df['income_group_id'] = df['total_income'].apply(income_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employment_info,purpose_id,age_group_id,income_group_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,available,2,3,3
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,available,3,3,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,available,2,3,2
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,available,4,3,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,available,1,3,2


Результирующая таблица, с которой можно работать и делать какие-то выводы, получена.

______________________

## Категоризация данных

Далее для удобства представления результатов выделим из таблицы основные данные и созадим несколько словарей.

- Финальную таблицу с данными назовем ***data***:

In [48]:
data = df[['children','dob_years','education_id','family_status_id','age_group_id','gender','total_income','purpose_id','income_group_id','debt']]
data.head()

,children,dob_years,education_id,family_status_id,age_group_id,gender,total_income,purpose_id,income_group_id,debt
0,1,42,0,0,3,F,253875,2,3,0
1,1,36,1,0,3,F,112080,3,2,0
2,0,33,1,0,3,M,145885,2,2,0
3,3,32,1,0,3,M,267628,4,3,0
4,0,53,1,1,3,F,158616,1,2,0


Получим словарь с типами образования, который назовем education_dict и преобразуем к финальному виду, удалив дубликаты.

In [49]:
education_dict = df[['education','education_id']]
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


- Словарь с типами образования ***education_dict***:

In [50]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Получим словарь с типами семйного статуса, который назовем family_status_dict и преобразуем к финальному виду, удалив дубликаты.

In [51]:
family_status_dict = df[['family_status','family_status_id']]
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


- Словарь с типами образования ***family_status_dict***:

In [52]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


- Словарь с возрастными группами ***age_group_dict*** был получен ранее: 

In [53]:
age_group_dict

,age_group,age_group_id
0,молодые,1
1,взрослые,2
2,пожилые,3


- Словарь с целями займа ***purpose_dict*** был получен ранее: 

In [54]:
purpose_dict

,target,purpose_id
0,свадьба,1
1,недвижимость,2
2,автомобиль,3
3,образование,4


- Словарь с уровнем дохода ***income_group_dict*** был получен ранее: 

In [55]:
income_group_dict

,income_group,income_group_id
0,низкий,1
1,средний,2
2,высокий,3


# Этап 3. Ответы на вопросы

### 1. Ответим на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?"

Чтобы ответить на этот вопрос, нам понадобится сводная таблица. В ней расчитаем суммарное количество должников ('sum','debt') среди общего количества людей ('len','debt') в каждой группе. Вычислим долю должников в группе в процентах ('debters_percentage','').

In [56]:
debt_from_children = data.pivot_table(index=['children'], values='debt', aggfunc=[sum,len])
debt_from_children['debters_percentage'] = (debt_from_children[('sum','debt')] / debt_from_children[('len','debt')]).round(3)*100
debt_from_children.sort_values(by='debters_percentage', ascending=False)

,sum,len,debters_percentage
,debt,debt,
children,,,
4,4,41,9.8
2,202,2128,9.5
1,445,4855,9.2
3,27,330,8.2
0,1063,14091,7.5
5,0,9,0.0


### Вывод 1.

Мы выяснили, что чем больше у заёмщика детей, тем больше среди них доля должников. Хотя выборка среди людей, имеющих по 3 и более детей не слишком невелика, можно сделать вывод о том, что чем меньше у заёмщика детей, тем меньше риск, что он не вернёт кредит в срок. Разница в доле должников заёмщиков без детей и заёмщиков, имеющим 1-2 ребенка - примерно 2%.  

### 2. Ответим на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?"

Аналогично ответу на 1-й вопрос, найдем процент должников среди людей с разным семейным положением.

In [57]:
debt_from_status = data.pivot_table(index=['family_status_id'], values='debt', aggfunc=[sum,len])
debt_from_status[('debters_percentage','')] = (debt_from_status[('sum','debt')] / debt_from_status[('len','debt')]).round(3)*100
debt_from_status=debt_from_status.sort_values(by='debters_percentage', ascending=False)
debt_from_status

,sum,len,debters_percentage
,debt,debt,
family_status_id,,,
4,274,2810,9.8
1,388,4151,9.3
0,931,12339,7.5
3,85,1195,7.1
2,63,959,6.6


Применим merge() чтобы объединенить полученную таблицу со словарём для большей наглядности. Несмотря на предупреждение о том, что при объединении таблиц с разным количеством уровней могут возникнуть непредвиденные результаты, выведем таблицу. Получекнный результат вполне нагляден и годится для анализа.

In [58]:
#debt_from_status.columns

In [59]:
debt_from_status=debt_from_status.merge(family_status_dict, on='family_status_id', how='right')
debt_from_status

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,family_status_id,"(sum, debt)","(len, debt)","(debters_percentage, )",family_status
0,4,274,2810,9.8,не женат / не замужем
1,1,388,4151,9.3,гражданский брак
2,0,931,12339,7.5,женат / замужем
3,3,85,1195,7.1,в разводе
4,2,63,959,6.6,вдовец / вдова


### Вывод 2.

Мы выяснили, что среди людей, не состоящих в браке, процент должников выше, чем среди женатых пар, примерно на 2%. В то же время пары, состоящие в браке опережают разведенных и вдовцов.

### 3. Ответим на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

Выясним, есть ли зависимость между уровнем дохода и возвратом кредита в срок среди людей, ежемесячный доход которых известен (не равен нулю).

In [60]:
#data_available = (data.loc[data.loc[:,'employment_info'] == 'available']).copy()
#data_available.head()
debt_from_income = data.loc[data.loc[:,'total_income'] !=0].pivot_table(index=['income_group_id'], values='debt', aggfunc=[sum,len])
debt_from_income[('debters_percentage','')] = (debt_from_income[('sum','debt')] / debt_from_income[('len','debt')]).round(3)*100
debt_from_income=debt_from_income.sort_values(by='debters_percentage', ascending=False)
debt_from_income

,sum,len,debters_percentage
,debt,debt,
income_group_id,,,
2,1137,13198,8.6
3,341,4824,7.1
1,93,1329,7.0


In [61]:
debt_from_income=debt_from_income.merge(income_group_dict, on='income_group_id', how='right')
debt_from_income

,income_group_id,"(sum, debt)","(len, debt)","(debters_percentage, )",income_group
0,2,1137,13198,8.6,средний
1,3,341,4824,7.1,высокий
2,1,93,1329,7.0,низкий


### Вывод 3.

Мы выяснили, что среди людей с низкой зарплатой должников меньше, чем со средней и высокой. 

### 4. Ответим на вопрос "Как разные цели кредита влияют на его возврат в срок?"

Выясним, есть ли зависимость между целью и возвратом кредита в срок.

In [62]:
debt_from_purpose = data.pivot_table(index=['purpose_id'], values='debt', aggfunc=[sum,len])
debt_from_purpose[('debters_percentage','')] = (debt_from_purpose[('sum','debt')] / debt_from_purpose[('len','debt')]).round(3)*100
debt_from_purpose=debt_from_purpose.sort_values(by='debters_percentage', ascending=False)
debt_from_purpose

,sum,len,debters_percentage
,debt,debt,
purpose_id,,,
3,403,4306,9.4
4,370,4013,9.2
1,186,2324,8.0
2,782,10811,7.2


In [63]:
debt_from_purpose=debt_from_purpose.merge(purpose_dict, on='purpose_id', how='right')
debt_from_purpose

,purpose_id,"(sum, debt)","(len, debt)","(debters_percentage, )",target
0,3,403,4306,9.4,автомобиль
1,4,370,4013,9.2,образование
2,1,186,2324,8.0,свадьба
3,2,782,10811,7.2,недвижимость


### Вывод 4.

Мы выяснили, что больше всего должников среди тех, кто берет кредит на автомобиль или образование. Среди тех, кто берет кредит на свадьбу, должников меньше. А меньше всего должников, не уложившихся в срок, среди тех, кто берет кредит на недвижимость. Это логично, т.к. цель долгосрочная, а деньги со временем обесцениваются, при том, что доход в валюте страны растет. Долгосрочные кредиты люди успевают гасить лучше, чем краткосрочные и среднесрочные.

# Этап 4. Общий вывод

Рабочие гипотезы:
- Наличие детей клиента влияет на факт погашения кредита в срок.
- Семейное положение клиента влияет на факт погашения кредита в срок.
- Уровень дохода влияет на факт погашения кредита в срок.
- Разные цели кредита влияют на его возврат в срок.


Результаты проверки гипотез:
- [x] Наличие детей клиента действительно влияет на факт погашения кредита в срок. Разница в доле должников заёмщиков без детей на 2% меньше доли заёмщиков, имеющих 1-2 детей.
- [x] Семейное положение клиента действительно влияет на факт погашения кредита в срок. Мы выяснили, что среди людей, не состоящих в браке, процент должников выше, чем среди женатых пар, примерно на 2%. В то же время пары, состоящие в браке опережают разведенных и вдовцов. Вероятно, это связано с тем, что не женатые/не замужние клиенты и состоящие в гражданском браке менее обремены обязательствами. Те, кто состоит в браке занимают золотую середину, вероятно потому, что у них есть поддержка партнёра по браку, и вдвоём выплачивать кредит проще. Меньше всего должников среди разведённых и вдовцов, вероятно потому, что разводятся и становятся вдовцами как правило становятся тогда, когда какие-то материальные потребности уже удовлетворены. Впрочем, не исключено, что при большей выборке все результаты больше стремились бы к среднему.
- [x] Уровень дохода действительно влияет на факт погашения кредита в срок. Мы выяснили, что среди людей со средним доходом самый высокий проценд должников. Вероятно потому, что их в принципе больше (есть потребность в ускорении достижения целей за счет кредита). Среди людей с высоким доходом процент должников меньше, видимо потому, что они могут позволить себе выплачивать кредит в срок, да и менее нуждаются в кредите. Среди людей с низким доходом доля должников почти такая же, что и с высоким доходом. Возможно, это объясняется тем, что у них каждая копейка на счету. При этом клиентов с низкой зарплатой меньше всего, возможно из-за низкого процента одобренных кредитов или нежелания влезать в долги.
- [x] Разные цели кредита действительно влияют на его возврат в срок. Среди заёмщиков, которые берут кредиты под долгосрочные цели, к которым, как правило, относится недвижимость, должников меньше всего. В то же время цель кредита на недвижимость преследует самая большая часть заёмщиков. Вероятно, это связано со сроками, на которые даётся кредит, и с высокой планкой для  одобрения кредита. Должников среди тех, кто берет кредит на автомобиль или образование, больше всего. Вероятно, потому, что они относятся относятся к категориям незамужних и со средним доходом.
